In [48]:
import json
from numpy import genfromtxt
from random import uniform, randint
import random as rand

def get_data():
    with open('6.txt') as file:
       max_weight, max_volume = [float(i) for i in next(file).split()]
       data = genfromtxt('6.txt', delimiter=' ', dtype=(int, float, int), skip_header=1)
    return {
        'max_weight': int(max_weight),
        'max_volume': float(max_volume),
        'items': data.tolist()
}

all_data = get_data()

Случайная генерация новой популяции

In [49]:
first_set = []
def rand_gen():
    for i in range(200):
        res=[]
        for i in range(0, 30):
            res.append(rand.randrange(2))
        first_set.append(res)
    return first_set

current_sets = rand_gen()

Выбор особи для скрещивания методом рулетки

In [50]:
def fitness(individual, data):
    weight, volume, price = 0, 0, 0
    for (selected, item) in zip(individual, data):
        if selected:
            weight += int(item[0])
            volume += int(item[1])
            price += int(item[2])
    if weight > all_data['max_weight'] or volume > all_data['max_volume']:
        price = 0
    return price

def get_max_fitness(sets):
    fitness_for_sets = []
    for j in range(len(sets)):
        fitness_for_sets.append(fitness(sets[j], all_data['items']))
    return max(fitness_for_sets)

def set_filter(sets):
    final_sets = []
    checking_percent = uniform(0, 1)
    fitness_for_sets = []
    for j in range(len(sets)):
        fitness_for_sets.append(fitness(sets[j], all_data['items']))
    max_fitness = max(fitness_for_sets)
    for i in range(len(fitness_for_sets)):
            fitness_for_sets[i] = fitness_for_sets[i] / max_fitness
            if checking_percent < fitness_for_sets[i]:
                final_sets.append(sets[i])
    return final_sets

Скрещивание (кроссинговер) между выбранными особями. Каждая особь скрещивается 1 раз за 1 поколение, 1 пара дает 2 потомка:3.1 многоточечный с 3мя точками

In [51]:
def cross(p1, p2):
    points = []
    for i in range(0, 3):
        points.append(randint(1, len(p1)))
    points.sort()
    ch1 = p1[:points[0]] + p2[points[0]:points[1]] + p1[points[1]:points[2]] + p2[points[2]:]
    ch2 = p2[:points[0]] + p1[points[0]:points[1]] + p2[points[1]:points[2]] + p1[points[2]:]
    return ch1, ch2

def set_cross(sets):
    child_sets = []
    for i in range(len(sets) - 1):
        child_sets += cross(sets[i], sets[i + 1])
    return child_sets

Мутация

In [52]:
def mutation(individual):
    count_zeroes = 0
    for i in range(len(individual)):
        if individual[i] == 0:
            count_zeroes += 1
    count_zeroes = int(count_zeroes * random())
    cur = 0
    for i in range(len(individual)):
        if individual[i] == 0:
            if cur == count_zeroes:
                individual[i] = 1
                return individual
            else:
                cur += 1
    return individual


def set_mutation(children):    
    sets = []
    final_len = round(len(sets) * 0.1)
    changing_sets = sets[:final_len]
    static_sets = sets[final_len:]
    for i in range(len(changing_sets)):
        sets.append(mutation(changing_sets[i]))
    return sets + static_sets

Формирование новой популяции (кол-во особей - константа): 5.1 замена не более 30% худших особей на потомков 

In [53]:
def set_new_population(children, sets, data):
    fitness_for_sets = []
    for i in range(len(sets)):
        fitness_for_sets.append(fitness(sets[i], data))
    fitnesses = {}
    for i in range(len(fitness_for_sets)):
        fitnesses[i] = fitness_for_sets[i]
    fitnesses = sorted(fitnesses.items(), key=lambda item: item[1])
    final_len = round(len(sets) * 0.3)
    for i in range(len(children) if len(children) < final_len else final_len):
        current_parent = fitnesses[i]
        sets[current_parent[0]] = children[i]
    return sets

In [54]:
current_sets = rand_gen()
max_fitness = get_max_fitness(current_sets)
current_count = 0

for i in range(500):
    filters_sets = set_filter(current_sets)
    children = set_cross(filters_sets)
    final_children = set_mutation(children)
    current_sets = set_new_population(final_children, current_sets, all_data['items'])
    prev_max_fitness = max_fitness
    max_fitness = get_max_fitness(current_sets)
    current_percent = abs((max_fitness - prev_max_fitness) / ((max_fitness + prev_max_fitness) / 2)) * 100
    if current_percent == 0:
        current_count += 1
    if current_count > 10:
        break
        
result_weight = 0
result_volume = 0
result_price = 0
result_sum = []
result_set = []
for i in range(len(current_sets)):
    fitness_for_set = fitness(current_sets[i], all_data['items'])
    if fitness_for_set == max_fitness:
        result_set = current_sets[i]
for i in range(len(result_set)):
    if result_set[i] > 0:
        result_weight += all_data['items'][i][0]
        result_volume += all_data['items'][i][1]
        result_price += all_data['items'][i][2]
        result_sum.append(i)
res = {
    'weight': result_weight,
    'volume': result_volume,
    'price': result_price,
    'items': result_sum
}
with open('result_2.json', 'w') as file:
    json.dump(res, file)